# 12 고급 pandas

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 12.1 Categorical 데이터

pandas의 `Categorical`형을 활용하여 pandas 메모리 사용량을 줄이고 성능을 개선할 수 있는 방법을 소개한다. 통계와 머신러닝에서 범주현 데이터를 활용하기 위한 도구들도 함께 소개!

### 12.1.1 개발 배경과 동기

하나의 컬럼에서 특정한 값이 반복되어 존재하는 경우는 흔하며 아래의 함수들을 통해 이들에 대한 정보를 확인할 수 있다.
- unique : 배열에서 존재하는 값의 목록
- value_counts : 배열에서 어떤 값이 몇개 들어있는가

In [3]:
import numpy as np; import pandas as pd
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values
pd.unique(values)
pd.value_counts(values)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

array(['apple', 'orange'], dtype=object)

apple     6
orange    2
dtype: int64

많은 데이터 시스템들은 이런 중복 데이터를 효율적으로 저장하고 계산할 수 있는가를 중점으로 개발되었다. 그 중 데이터웨어하우스라는 데이터 시스템에서는 실제 값을 나타내는 **차원 테이블**과 그 테이블을 참고하는 정수키를 사용한다.

- 정수로 표현된 값 👉 **범주형, 사전형 표기법** 👉 `values`
- 차원 테이블 👉 **범주, 사전, 단계 데이터** 👉 `dim`
- 범주형 테이터를 가리키는 정수값 : **범주코드** 👉 `values`의 원소

In [14]:
dim = pd.Series(['apple', 'orange']) # 차원 테이블
values = pd.Series([0, 1, 0, 0] * 2) # 정수키
dim
values

0     apple
1    orange
dtype: object

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [16]:
# 키를 활용하여 원래 값 복구
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

📌 Series.take
![image](https://user-images.githubusercontent.com/16831323/130546471-98288d35-e13d-4ee2-b6d1-ce7f1d408947.png)


In [18]:
df = pd.DataFrame([('falcon', 'bird', 389.0),
                   ('parrot', 'bird', 24.0),
                   ('lion', 'mammal', 80.5),
                   ('monkey', 'mammal', np.nan)],
                  columns=['name', 'class', 'max_speed'],
                  index=[0, 2, 3, 1])
df
df.take([0, 2], axis=0)
df.take([0, 2], axis=1)

,name,class,max_speed
0,falcon,bird,389.0
2,parrot,bird,24.0
3,lion,mammal,80.5
1,monkey,mammal,NaN


,name,class,max_speed
0,falcon,bird,389.0
3,lion,mammal,80.5


,name,max_speed
0,falcon,389.0
2,parrot,24.0
3,lion,80.5
1,monkey,NaN


이런한 범주형 표기법을 사용하면 분석 작업에서의 엄청난 성능 향상을 얻을 수 있다.

😎 범주 코드를 변경하지 않는채로 범주형 데이터를 변형하는 것도 가능<br>
- 범주형 데이터의 이름 변경하기
- 기존 범주형 데이터의 순서를 바꾸지 않고 새로운 범주 추가하기

### 12.1.2 pandas의 Categorical Type
정수 기반의 범주형 데이터를 표현(또는 **인코딩**)할 수 있는 `Categorical`형이라고 하는 특수한 데이터형이 존재한다.

In [19]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [21]:
# 문자열 객체 배열 👉 범주형 데이터
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [25]:
# fruit_cat의 값은 NumPy 배열이 아니라 pandas.Categorical의 인스턴스이다
c = fruit_cat.values
c
type(c)

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

pandas.core.arrays.categorical.Categorical

- `Categorical` 객체는 `categories`와 `codes` 속성을 가진다.

In [26]:
c.categories # The categories of this categorical.
c.codes      # The category codes of this categorical.
c.ordered    # Whether the categories have an ordered relationship.

Index(['apple', 'orange'], dtype='object')

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

False

In [32]:
# df['fruit']을 categorical로 변경하여 대입
df['fruit'] = df['fruit'].astype('category')
df
df.info()

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   basket_id  8 non-null      int32   
 1   fruit      8 non-null      category
 2   count      8 non-null      int32   
 3   weight     8 non-null      float64 
dtypes: category(1), float64(1), int32(2)
memory usage: 388.0 bytes


#### Sequential data → pandas.Categorical

![image](https://user-images.githubusercontent.com/16831323/130558504-bd9647b5-7614-4358-b7dd-85b915b0fe3a.png)

In [33]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

- 변수로 정의된 범주와 범주 코드를 이용하여 `Categorical`을 생성하는 것도 가능하다 👉 `from_codes`

![image](https://user-images.githubusercontent.com/16831323/130566277-9540a01f-0c68-4999-bab5-73706f94419b.png)


In [34]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [41]:
# 특정 순서를 보장하기 위해서는 ordered 파라미터를 사용
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat # 순서가 foo → bar → baz
ordered_cat.ordered

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

True

- 순서가 없는 범주형 인스턴스 → 순서가 잇는 범주형 인스턴스 : `as_ordered`

In [36]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

예시를 문자열로만 들었었는데 범주형이 **꼭** 문자열일 필요는 없다. 범주형 배열은 변경이 불가능한 값이라면 <u>어떤</u> 자료형이라도 포함할 수 있다.

In [43]:
categories = [1, 2, 3]
codes = [0, 1, 2, 0, 0, 1]
num_cats = pd.Categorical.from_codes(codes, categories)
num_cats

[1, 2, 3, 1, 1, 2]
Categories (3, int64): [1, 2, 3]

### 12.1.3 Categorical 연산
- pandas에서 `Categorical`은 문자열 배열처럼 인코딩되지 않은 자료형을 사용하는 방식과 거의 유사하게 사용할 수 있다.

#### 연산 예시 : qcut
![image](https://user-images.githubusercontent.com/16831323/130571415-c08bea9a-3f9e-439a-b11b-bca1bafc3e14.png)

In [44]:
np.random.seed(12345)
draws = np.random.randn(1000) # 임의의 숫자 데이터
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [53]:
bins = pd.qcut(draws, 4) # 임의의 숫자 데이터를 사분위로 나눈다.
bins # Categorical이 반환된다.
bins.codes[:10]

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [56]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4']) # 라벨 추가
bins
bins.codes[:10]

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [59]:
bins = pd.Series(bins, name='quartile')

# 라벨을 붙이고 나면 데이터의 시작과 끝에 대한 정보를 포함하지 못하게 되므로
# groupby를 이용해서 요약 통계를 내보자👊
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [60]:
# 이때, quartile 컬럼의 bins의 순서를 포함한 원래 범주 정보를
# 잘 유지하고 있다.
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### categorical을 이용한 선능 개선
특정 데이터셋 대해 다양한 분석을 하는 경우 범주형으로 변환하는 것만으로도 전체 성능을 개선할 수 있다.<br>
👉 범주형에 대한 그룹 연산은 정수 기반의 코드 배열을 사용한다.<br>
👉 연산 속도가 빠르며 메모리 역시 더 적게 사용한다.

In [65]:
N = 10000000

# 소수의 독립적인 카테고리로 분류디는 천만 개의 값을 포함하는 Series
draws = pd.Series(np.random.randn(N))

# Series → categorical
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')
categories

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: category
Categories (4, object): ['bar', 'baz', 'foo', 'qux']

In [66]:
# 메모리 사용량 비교
labels.memory_usage()     # 80000128
categories.memory_usage() # 10000332

80000128

10000332

In [67]:
import sys
sys.getsizeof('1')
sys.getsizeof(1)
sys.getsizeof(True)

50

28

28

In [64]:
# 범주형으로 변환하는 과정 역시 비용이 들기는 하지만 일회성이기 때문에 
# 변환을 한 후 연산과정에서 범주형이 가지는이득이 
# 변환 비용보다 더 크다😋
%time _ = labels.astype('category')

Wall time: 1.86 s


### 12.1.4 Categorical 메서드
범주형 데이터를 담고 있는 Series가 가지는 특수한 메서드 몇가지를 알아보자👊

In [69]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [76]:
cat_s._get_values

<bound method Series._get_values of 0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']>

In [77]:
cat_s.cat.codes
cat_s.cat.categories

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

Index(['a', 'b', 'c', 'd'], dtype='object')

- 특별한 속성인 `cat`를 통해 `categorical` 메서드에 접근할 수 있다.

In [79]:
help(pd.Series.cat)

Help on class CategoricalAccessor in module pandas.core.arrays.categorical:

class CategoricalAccessor(pandas.core.accessor.PandasDelegate, pandas.core.base.PandasObject, pandas.core.base.NoNewAttributesMixin)
 |  CategoricalAccessor(data)
 |  
 |  Accessor object for categorical properties of the Series values.
 |  
 |  Be aware that assigning to `categories` is a inplace operation, while all
 |  methods return new categorical data per default (but can be called with
 |  `inplace=True`).
 |  
 |  Parameters
 |  ----------
 |  data : Series or CategoricalIndex
 |  
 |  Examples
 |  --------
 |  >>> s = pd.Series(list("abbccc")).astype("category")
 |  >>> s
 |  0    a
 |  1    b
 |  2    b
 |  3    c
 |  4    c
 |  5    c
 |  dtype: category
 |  Categories (3, object): ['a', 'b', 'c']
 |  
 |  >>> s.cat.categories
 |  Index(['a', 'b', 'c'], dtype='object')
 |  
 |  >>> s.cat.rename_categories(list("cba"))
 |  0    c
 |  1    b
 |  2    b
 |  3    a
 |  4    a
 |  5    a
 |  dtype: categ

- 메서드 및 멤버변수의 종류는 다음과 같다.
| 이름 | 내용 |
|:-----:|:-----|
| categories | The categories of this categorical. |
| ordered | Whether the categories have an ordered relationship. |
| codes | Return Series of codes as well as the index. |
| rename_categories | Rename categories. 카테고리 이름을 지정한 이름으로 변경한다. 카테고리 수는 변하지 않는다. |
| reorder_categories | Reorder categories as specified in new_categories. rename_categories와 유사하지만 새로운 카테고리가 순서를 가지도록 한다. |
| add_categories | Add new categories. 기존 카테고리 끝에 새로운 카테고리를 추가. |
| remove_categories | Remove the specified categories. 카테고리를 제거한다. 해당 카테고리에 속한 값들은 null로 설정한다. |
| remove_unused_catego | Remove categories which are not used. 데이터에서 관측되지 않는 카테고리를 삭제한다. |
| set_categories | Set the categories to the specified new_categories. 카테고리를 지정한 새로운 카테고리로 변경한다. 카테고리 추가나 삭제가 가능하다. |
| as_ordered | Set the Categorical to be ordered. 카테고리를 순서를 가지도록 한다. |
| as_unordered | Set the Categorical to be unordered. 카테고리가 순서를 가지지 않도록 한다. |

- set_categories : 카테고리 변경

In [81]:
# 해당 데이터의 실제 카테고리가 4종류 이상이라고 가정하면
# 카테고리를 actual_categories로 변경하고자 한다.
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [82]:
# 변경 전후의 카테고리면 원소갯수
cat_s.value_counts()
cat_s2.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

a    2
b    2
c    2
d    2
e    0
dtype: int64

- remove_unused_categories : 실제로 관측되지 않는 카테고리 제거

In [83]:
# cat에서 a, b 값을 가지는 데이터를 추출
cat_s3 = cat_s[cat_s.isin(['a', 'b'])] 
cat_s3
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

#### 모델링을 위한 더미값 생성하기 
👉 원핫인코딩

In [84]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [85]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
